In [1]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
print(cv2.__version__)

#Pop-up do Matplotlib
#%matplotlib qt

#Exibição na mesma tela do Jupyter
%matplotlib inline

#plt.rcParams["figure.figsize"]\
#(8,5)


from io import BytesIO
from IPython.display import clear_output, Image, display
import PIL



plt.style.use('seaborn')
sns.set_style("whitegrid", {'axes.grid' : False})

3.4.2


/Users/michel.fernandes/anaconda/lib/python3.6/site-packages/joblib/_multiprocessing_helpers.py:28: UserWarning: [Errno 28] No space left on device.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [2]:
classes = None

with open("data/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
    
cores_randomicas_classes = np.random.uniform(0, 255, size=(len(classes), 3))

len(classes)

net = cv2.dnn.readNet("pesos/yolov3.weights", "cfg/yolov3.cfg")

In [3]:
def obter_camadas_saida(net):
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

In [ ]:


def desenhar_predicao(img, class_id, confidence, x, y, x_plus_w, y_plus_h):
    label = str(classes[class_id])
    color = cores_randomicas_classes[class_id]
    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)
    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return img

In [ ]:
imagem = cv2.imread("imagens/banana.jpeg")
imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
(width, height) = imagem.shape[:2]
escala = 1 / 255.0
#imagem = cv2.resize(imagem, (416, 416))

In [ ]:
blob = cv2.dnn.blobFromImage(imagem, 1 / 255.0, (416, 416), (0,0,0), swapRB=False, crop=False)
net.setInput(blob)
outs = net.forward(obter_camadas_saida(net))

In [ ]:
len(outs), len(blob), net, len(classes)

In [39]:
class_ids = []
confidences = []
boxes = []

conf_threshold = 0.7
nms_threshold = 0.4

In [5]:
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = center_x - w / 2
            y = center_y - h / 2
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, w, h])

indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)



for i in indices:
    i = i[0]
    box = boxes[i]
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    
    label = str(classes[class_ids[i]]) + " " + str(confidences[i])
    color = cores_randomicas_classes[class_ids[i]]
    cv2.rectangle(imagem, (round(x),round(y)), (round(x+w),round(y+h)), color, 2)
    cv2.putText(imagem, label, (round(x)-10,round(y)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    #imagem = desenhar_predicao(imagem, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))

#imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(20,10))
plt.imshow(imagem)
plt.title("Teste")

NameError: name 'outs' is not defined

In [45]:
def identificar_objeto(imagem):
    (width, height) = imagem.shape[:2]
    blob = cv2.dnn.blobFromImage(imagem, 1/255.0, (416,416), True, crop=False)
    net.setInput(blob)
    outs = net.forward(obter_camadas_saida(net))

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

    for i in indices:
        i = i[0]
        box = boxes[i]
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
        #imagem = desenhar_predicao(imagem, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))
        
        label = str(classes[class_ids[i]]) + " " + str(confidences[i])
        color = cores_randomicas_classes[class_ids[i]]
        cv2.rectangle(imagem, (round(x),round(y)), (round(x+w),round(y+h)), color, 2)
        cv2.putText(imagem, label, (round(x)-10,round(y)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    #imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
        
    return imagem

In [49]:
#cam.release()
cam = cv2.VideoCapture("videos/cars.avi")

try:
    while(True):
        is_capturing, frame = cam.read()
        if is_capturing:
            
            (width, height) = frame.shape[:2]
            blob_frame = cv2.dnn.blobFromImage(frame, 1/255.0, (416,416), True, crop=False)
            net.setInput(blob_frame)
            outs = net.forward(obter_camadas_saida(net))

            for out in outs:
                for detection in out:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > 0.5:
                        center_x = int(detection[0] * width)
                        center_y = int(detection[1] * height)
                        w = int(detection[2] * width)
                        h = int(detection[3] * height)
                        x = center_x - w / 2
                        y = center_y - h / 2
                        class_ids.append(class_id)
                        confidences.append(float(confidence))
                        boxes.append([x, y, w, h])

            indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

            for i in indices:
                i = i[0]
                box = boxes[i]
                x = box[0]
                y = box[1]
                w = box[2]
                h = box[3]
               
                label = str(classes[class_ids[i]]) + " " + str(confidences[i])
                color = cores_randomicas_classes[class_ids[i]]
                cv2.rectangle(frame, (round(x),round(y)), (round(x+w),round(y+h)), color, 2)
                cv2.putText(frame, label, (round(x)-10,round(y)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            
            img = PIL.Image.fromarray(frame, "RGB")
            buffer = BytesIO()
            img.save(buffer,format="JPEG")        
            display(Image(data=buffer.getvalue()))
            
            #cv2.imwrite("imagens/camera_output_3.png", frame)

        clear_output(wait=True)
except KeyboardInterrupt:
    cam.release()
    print("Interrompido")

Interrompido
